<a href="https://colab.research.google.com/github/AffanSurya/SenakApp_ML/blob/main/Model/RecomandationsS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.0 MB/s eta 0:00:00


In [2]:
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_recipes = pd.read_csv('/content/drive/MyDrive/Dataset/recipes_reduce.csv')
df_reviews = pd.read_csv('/content/drive/MyDrive/Dataset/reviews_reduce.csv')

In [5]:
df_recipes.info()
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27242 entries, 0 to 27241
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   RecipeId                    27242 non-null  int64  
 1   Name                        27242 non-null  object 
 2   AuthorId                    27242 non-null  int64  
 3   Description                 27242 non-null  object 
 4   Images                      27242 non-null  object 
 5   RecipeCategory              27232 non-null  object 
 6   RecipeIngredientQuantities  27242 non-null  object 
 7   RecipeIngredientParts       27242 non-null  object 
 8   AggregatedRating            27241 non-null  float64
 9   ReviewCount                 27242 non-null  float64
 10  Calories                    27242 non-null  float64
 11  Fat                         27242 non-null  float64
 12  SaturatedFat                27242 non-null  float64
 13  Cholesterol                 272

In [6]:
df_recipes.head()

,RecipeId,Name,AuthorId,Description,Images,RecipeCategory,RecipeIngredientQuantities,RecipeIngredientParts,AggregatedRating,ReviewCount,Calories,Fat,SaturatedFat,Cholesterol,Sodium,Carbohydrate,Fiber,Sugar,Protein,RecipeInstructions
0,40,Best Lemonade,1566,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",Beverages,"c(""1 1/2"", ""1"", NA, ""1 1/2"", NA, ""3/4"")","c(""sugar"", ""lemons, rind of"", ""lemon, zest of""...",4.5,10.0,311.1,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,"c(""Into a 1 quart Jar with tight fitting lid, ..."
1,42,Cabbage Soup,1538,Make and share this Cabbage Soup recipe .,"""https://img.sndimg.com/food/image/upload/w_55...",Vegetable,"c(""46"", ""4"", ""1"", ""2"", ""1"")","c(""plain tomato juice"", ""cabbage"", ""onion"", ""c...",4.5,11.0,103.6,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,"c(""Mix everything together and bring to a boil..."
2,44,Warm Chicken A La King,1596,I copied this one out of a friend's book so ma...,"""https://img.sndimg.com/food/image/upload/w_55...",Chicken,"c(""12"", ""2"", ""3"", ""450"", ""1"", ""2"", ""1/4"", ""1"",...","c(""chicken"", ""butter"", ""flour"", ""milk"", ""celer...",5.0,23.0,895.5,66.8,31.9,405.8,557.2,29.1,3.1,5.0,45.3,"c(""Melt 1 1/2 ozs butter, add the flour and co..."
3,49,Chicken Breasts Lombardi,174711,Make and share this Chicken Breasts Lombardi r...,"c(""https://img.sndimg.com/food/image/upload/w_...",Chicken Breast,"c(""2"", ""2"", ""12"", ""1/2"", ""1/3"", ""3/4"", ""1/2"", ...","c(""fresh mushrooms"", ""butter"", ""boneless skinl...",5.0,21.0,627.7,24.9,13.0,203.0,848.9,13.7,0.6,2.0,57.9,"c(""Cook mushrooms in 2 tbsp butter in a large ..."
4,54,Carrot Cake,1535,This is one of the few recipes my husband ever...,"c(""https://img.sndimg.com/food/image/upload/w_...",Dessert,"c(""3"", ""4"", ""1 1/4"", ""2"", ""2"", ""2"", ""2"", ""1"", ...","c(""carrots"", ""eggs"", ""white sugar"", ""all-purpo...",5.0,17.0,522.6,27.1,4.9,69.8,534.8,67.0,1.6,47.9,5.0,"c(""Beat together the eggs, oil, and white suga..."


In [7]:
df_reviews.head()

,ReviewId,Name,RecipeId,AuthorId,Rating,Review,DateSubmitted,DateModified
0,2,Jalapeno Pepper Poppers,992,2008,5,better than any you can get at a restaurant!,2000-01-25T21:44:00Z,2000-01-25T21:44:00Z
1,9,Chinese Imperial Palace General Tso's Chicken,4523,2046,2,i think i did something wrong because i could ...,2000-02-25T09:00:00Z,2000-02-25T09:00:00Z
2,13,Kevin's Best Corned Beef,7435,1773,5,easily the best i have ever had. juicy flavor...,2000-03-13T21:15:00Z,2000-03-13T21:15:00Z
3,14,Warm Chicken A La King,44,2085,5,An excellent dish.,2000-03-28T12:51:00Z,2000-03-28T12:51:00Z
4,17,Spinach and Strawberry Salad,5221,2046,4,"love it, but without the bean sprouts.",2000-05-08T11:08:00Z,2000-05-08T11:08:00Z


In [9]:
df_reviews = df_reviews.astype({'AuthorId': 'string'})


Reviews = tf.data.Dataset.from_tensor_slices(dict(df_reviews[['AuthorId', 'Name', 'Rating']]))
Recipes = tf.data.Dataset.from_tensor_slices(dict(df_recipes[['Name']]))

Reviews = Reviews.map(lambda x: {
    "Name": x["Name"],
    "AuthorId": x["AuthorId"],
    "Rating": float(x["Rating"])
})

Recipes = Recipes.map(lambda x: x["Name"])

In [10]:
print('Total Data: {}'.format(len(Reviews)))

tf.random.set_seed(42)
shuffled = Reviews.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = Reviews.take(35_000)
test = Reviews.skip(35_000).take(8_188)

Total Data: 787352


In [11]:
food_titles = Recipes.batch(1_000)
user_ids = Reviews.batch(1_000).map(lambda x: x["AuthorId"])

unique_food_titles = np.unique(np.concatenate(list(food_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print('Unique Recipes: {}'.format(len(unique_food_titles)))
print('Unique users: {}'.format(len(unique_user_ids)))

Unique Recipes: 26298
Unique users: 195336


In [13]:
class FoodModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 64

    # User and movie models.
    self.food_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_food_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_food_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=Recipes.batch(128).map(self.food_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["AuthorId"])
    # And pick out the movie features and pass them into the movie model.
    food_embeddings = self.food_model(features["Name"])

    return (
        user_embeddings,
        food_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, food_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    ratings = features.pop("Rating")

    user_embeddings, food_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, food_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)

In [ ]:
# Convert a batch of the train dataset to a Pandas DataFrame
train_batch = next(iter(train.batch(5)))  # You can adjust the batch size as needed
train_df = pd.DataFrame(train_batch)

# Print the data type and unique values of the "Rating" column
print(train_df["Rating"].dtype)
print(train_df["Rating"].unique())


float32
[5. 4. 2.]


In [ ]:
# Print data types and unique values of all columns in train_df
for column in train_df.columns:
    print(f"{column}: {train_df[column].dtype}, Unique values: {train_df[column].unique()}")


RecipeId: object, Unique values: [b'992' b'4384' b'4523' b'7435' b'44']
AuthorId: object, Unique values: [b'2008' b'1634' b'2046' b'1773' b'2085']
Rating: float32, Unique values: [5. 4. 2.]


In [14]:
model = FoodModel(rating_weight=1.0, retrieval_weight=1.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

cached_train = train.shuffle(100_000).batch(1_000).cache()
cached_test = test.batch(1_000).cache()

model.fit(cached_train, epochs=3)

Epoch 1/3
35/35 [==============================] - 129s 4s/step - root_mean_squared_error: 1.6512 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0017 - factorized_top_k/top_10_categorical_accuracy: 0.0038 - factorized_top_k/top_50_categorical_accuracy: 0.0129 - factorized_top_k/top_100_categorical_accuracy: 0.0196 - loss: 6909.0195 - regularization_loss: 0.0000e+00 - total_loss: 6909.0195
Epoch 2/3
35/35 [==============================] - 124s 4s/step - root_mean_squared_error: 1.1048 - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0546 - factorized_top_k/top_10_categorical_accuracy: 0.0850 - factorized_top_k/top_50_categorical_accuracy: 0.2026 - factorized_top_k/top_100_categorical_accuracy: 0.2877 - loss: 6723.8141 - regularization_loss: 0.0000e+00 - total_loss: 6723.8141
Epoch 3/3
35/35 [==============================] - 123s 3s/step - root_mean_squared_error: 1.1386 - 

In [15]:
metrics = model.evaluate(cached_test, return_dict=True)

print(f"\nRetrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}")

9/9 [==============================] - 38s 4s/step - root_mean_squared_error: 0.8481 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0011 - factorized_top_k/top_10_categorical_accuracy: 0.0022 - factorized_top_k/top_50_categorical_accuracy: 0.0103 - factorized_top_k/top_100_categorical_accuracy: 0.0200 - loss: 5826.1750 - regularization_loss: 0.0000e+00 - total_loss: 5826.1750

Retrieval top-100 accuracy: 0.020
Ranking RMSE: 0.848


In [58]:

random_user_id = df_reviews['AuthorId'].sample(1).values[0]

def predict_food_for_random_user(top_n=5):
    index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
    index.index_from_dataset(
      tf.data.Dataset.zip((Recipes.batch(100), Recipes.batch(100).map(model.food_model)))
    )

    _, titles = index(tf.constant([str(random_user_id)]))

    print('Top {} recommendations for user {}:\n'.format(top_n, random_user_id))
    for i, title in enumerate(titles[0, :top_n].numpy()):
        print('{}. {}'.format(i+1, title.decode("utf-8")))

predict_food_for_random_user(top_n=5)


Top 5 recommendations for user 1041521:

1. Chicken Stroganoff - Crock Pot
2. Creamy Bacon-Ranch Pasta Salad
3. Cracker Barrel's Hashbrowns Casserole - Copycat
4. Still Another Tuna/Pasta Salad.......(Mom's Tuna Salad)
5. Pork Chops In Gravy


In [34]:
# predict_food(2, 5)

Top 5 recommendations for user 2:

1. Easy Breakfast Casserole
2. Mexican Vegetarian Casserole
3. Green Beans with Cherry Tomatoes
4. Buttered Baby Carrots and Sweet Peas
5. Steven's World Famous To-Die-For Sour Cream Chicken
